### EDA for ELO Customer Transactions

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
import pandas_profiling
import warnings
warnings.filterwarnings('ignore')

In [ ]:

pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_columns', None)

In [ ]:
#loading csv
hist_transactions = pd.read_csv('E:/ELOData/historical_transactions.csv',parse_dates=['purchase_date'])
hist_transactions.head()

In [ ]:
#loading csv
new_merchant_transactions = pd.read_csv('E:/ELOData/new_merchant_transactions.csv',parse_dates=['purchase_date'])
new_merchant_transactions.head()

In [ ]:
pandas_profiling.ProfileReport(hist_transactions)
#Duplicate card ids which is expected as there are multiple transactions
#category_2 has missing values 

In [ ]:
pandas_profiling.ProfileReport(new_merchant_transactions)

## Looking at the profiling report and the correlation matrix here is what our next steps are : 


### There are missing values as NaNs , we try to impute them as below:

* for category_3 we see that most of the observations fall in the category value A , hence we use that as our imputed value such that the distribution is not hampered.
* for category_2 we see that most of the observations fall in the category value 1 , hence we use that as our imputed value such that the distribution is not hampered.
    
### Few columns have NaNs as values , which we dont impute or delete :

* merchant_id field has missing values , however we dont drop those rows as they may beimportant from transactions per-say , we dont want to impute them as well 

#### We observe the columns like Installements and Purchase Amounts that are highly skewed
#### We also do some EDA to check for cards who have purchased subscription based products like Netflix etc
#### Looking at the correlation plots and correlation values for historical and new , we dont see very high correlation amonst the variables on both the data frames


In [ ]:
hist_transactions.columns[hist_transactions.isna().any()].tolist()

In [ ]:
sns.heatmap(hist_transactions.corr())


In [ ]:
sns.heatmap(new_merchant_transactions_transactionstransactionstransactions.corr())

In [ ]:
corrmat = hist_transactions.corr()
k = 5 #Nor depth has any deep correlation with any other variable
cols = corrmat.nlargest(k, 'purchase_amount')['purchase_amount'].index
print(cols)
fig = plt.figure(num=None, figsize=(12, 8)) 
cm = np.corrcoef(hist_transactions[cols].values.T)
sns.set(font_scale=1.25)
sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
#corrmat#.style.background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)#format("{:.2}")#
hist_transactions.corr()

In [ ]:
hist_transactions[hist_transactions['merchant_category_id'].isnull()==True].shape

In [ ]:
hist_transactions.describe()

In [ ]:
print(hist_transactions['category_2'].value_counts(dropna=False))
print(new_merchant_transactions['category_2'].value_counts(dropna=False))

In [ ]:
print(hist_transactions.shape)
print(new_merchant_transactions.shape)

In [ ]:
hist_transactions.columns[hist_transactions.isna().any()].tolist()

In [ ]:
new_merchant_transactions.columns[new_merchant_transactions.isnull().any()].tolist()

In [ ]:
print(hist_transactions['authorized_flag'].value_counts(dropna=False))
print(new_merchant_transactions['authorized_flag'].value_counts(dropna=False))

In [ ]:
print(hist_transactions['category_1'].value_counts(dropna=False))
print(new_merchant_transactions['category_1'].value_counts(dropna=False))

In [ ]:
print(hist_transactions['purchase_date'].min())
print(hist_transactions['purchase_date'].max())
print(new_merchant_transactions['purchase_date'].min())
print(new_merchant_transactions['purchase_date'].max())

In [ ]:
hist_transactions.fillna({x:1.0 for x in ['category_2']}, inplace=True)
new_merchant_transactions.fillna({x:1.0 for x in ['category_2']}, inplace=True)

In [ ]:
hist_transactions.columns[hist_transactions.isna().any()].tolist()

In [ ]:
print(hist_transactions['category_3'].value_counts(dropna=False))
print(new_merchant_transactions['category_3'].value_counts(dropna=False))

In [ ]:
hist_transactions.fillna({x:'A' for x in ['category_3']}, inplace=True)
new_merchant_transactions.fillna({x:'A' for x in ['category_3']}, inplace=True)

In [ ]:
#We will not drop these records and later deal with these while modelling/aggregating
print(hist_transactions[hist_transactions['merchant_id'].isna()==True].shape)
print(new_merchant_transactions[new_merchant_transactions['merchant_id'].isna()==True].shape)

In [ ]:
print(hist_transactions.columns[hist_transactions.isna().any()].tolist())
print(new_merchant_transactions.columns[new_merchant_transactions.isna().any()].tolist())


In [ ]:
hist_transactions[hist_transactions==np.inf]=np.nan
new_merchant_transactions[hist_transactions==np.inf]=np.nan

In [ ]:
print(hist_transactions.columns[hist_transactions.isna().any()].tolist())
print(new_merchant_transactions.columns[new_merchant_transactions.isna().any()].tolist())

In [ ]:
hist_transactions[hist_transactions['purchase_amount'] <0 ].shape

In [ ]:
hist_transactions[hist_transactions['purchase_amount'] >0 ].shape

In [ ]:
print(hist_transactions.shape)
print(new_merchant_transactions.shape)

#### Trying a few different transformations on the variables of interest to see if they normalize after transformations

In [ ]:
#This block considers all variables and can be used iteratively to minimize re-running code
#We can limit the variables of interest if we want to
def Plot(varname,dataset):   
    f, (ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(20,4))
    
    ax1.set_title('Before Log for var:' + varname)
    ax2.set_title('After Log for var:' + varname)
    ax3.set_title('After Sqrt for var:' + varname) 
    
    sns.distplot(dataset[varname].fillna(0),ax=ax1,fit=norm)   
    sns.distplot((np.log(dataset[varname]).fillna(0)),ax=ax2,fit=norm)
    sns.distplot((np.sqrt(dataset[varname]).fillna(0)),ax=ax3,fit=norm)    
    #f.savefig(varname+".png")
    plt.show()
   

In [ ]:
Plot('merchant_category_id',hist_transactions)

In [ ]:
Plot('category_2',hist_transactions)

In [ ]:
Plot('city_id',hist_transactions)

In [ ]:
Plot('subsector_id',hist_transactions)

In [ ]:
Plot('state_id',hist_transactions)

In [ ]:
#loading csv
merchants = pd.read_csv('E:/ELOData/merchants_transformed.csv')
merchants.head() 

In [ ]:
hist_transactions.sort_values(['card_id','merchant_id']).head(5)

In [ ]:
hist_transactions[hist_transactions['merchant_id']=='M_ID_ebbdb42da6'].sort_values(['card_id']).head(10)

In [ ]:
merchants[merchants['merchant_id']=='M_ID_ebbdb42da6'].head()

In [ ]:
hist_transactions[hist_transactions['merchant_id']=="M_ID_fc7d7969c3"].purchase_amount.value_counts()

In [ ]:
hist_transactions[(hist_transactions['merchant_id']=="M_ID_fc7d7969c3")]

In [ ]:
merchants[merchants['merchant_category_id']==661].head(10)

In [ ]:
new_merchant_transactions[new_merchant_transactions['merchant_id']=="M_ID_fc7d7969c3"].purchase_amount.value_counts()

In [ ]:
new_merchant_transactions[new_merchant_transactions['merchant_id']=="M_ID_fc7d7969c3"].purchase_amount.value_counts()

In [ ]:
hist_transactions[ (hist_transactions['card_id']=='C_ID_989c3d4981')].sort_values('purchase_date',ascending=True)['purchase_amount'].std()

In [ ]:
new_merchant_transactions[new_merchant_transactions['card_id']=='C_ID_989c3d4981'].head()

In [ ]:
#C_ID_bbc950fc99/C_ID_ba98edb1f6
print(hist_transactions[(hist_transactions['authorized_flag'] == 'Y') & (hist_transactions['card_id']=='C_ID_ba98edb1f6')]['purchase_amount'].sum())
print(#C_ID_bbc950fc99/C_ID_ba98edb1f6
hist_transactions[(hist_transactions['authorized_flag'] == 'Y') & (hist_transactions['card_id']=='C_ID_bbc950fc99')]['purchase_amount'].sum())
print(hist_transactions[(hist_transactions['authorized_flag'] == 'Y') & (hist_transactions['card_id']=='C_ID_176ed894ed')]['purchase_amount'].sum())

print(hist_transactions[(hist_transactions['authorized_flag'] == 'Y') & (hist_transactions['card_id']=='C_ID_989c3d4981')]['purchase_amount'].sum())




In [ ]:
cards = hist_transactions[hist_transactions['merchant_id']=="M_ID_fc7d7969c3"]['card_id'].value_counts()

In [ ]:
hist_transactions[hist_transactions['card_id'].isin(cards.index.values)].groupby(['card_id','merchant_id']).size().reset_index()

In [ ]:
hist_transactions[hist_transactions['card_id'].isin(cards.index.values)].groupby(['card_id']).merchant_id.nunique().sort_values(ascending=True)

In [ ]:
hist_transactions.to_csv('/Users/rajmati.marlecha/Documents/elo-merchant-category-recommendation/hist_trans_transformed.csv', index = False, header = True)
new_merchant_transactions.to_csv('/Users/rajmati.marlecha/Documents/elo-merchant-category-recommendation/new_trans_transformed.csv', index = False, header = True)